In [2]:
import pandas as pd
import numpy as np
import bitcoin
import datetime
from ping3 import ping
import random
import json
import re
import multiprocess as mp

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'e:\\Codes\\PythonWorkspace\\CSE5005-Project\\data'

In [3]:
with open("./website.json", "r", encoding="utf8") as f:
    website_list = json.load(f)

website_list[:10]

[{'id': '2205',
  'name': '人人网',
  'logo': 'http://reg007.u.qiniudn.com/app/renren.jpg',
  'desc': '人人网，中国领先的实名制SNS社交网络。加入人人网，找到老同学，结识新朋友。',
  'category': '社交',
  'home': 'http://www.renren.com/'},
 {'id': '2217',
  'name': '途牛旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/tuniu.jpg',
  'desc': '途牛旅游网-中国专业全面的旅游网,客户满意度94%;提供自助游(自由行),国内旅游,出境旅游,自驾游,公司旅游。低价保证,专业服务,九大出游保障,服务超百万人出游。',
  'category': '旅行',
  'home': 'http://www.tuniu.com/'},
 {'id': '2224',
  'name': '酷讯旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/kuxun.jpg',
  'desc': '酷讯旅游网(Kuxun.cn)是中国领先的在线旅游媒体。为您提供低价可靠的国内国际机票，特价机票99元起，另提供全国20万家酒店预订查询、酒店评论，最低2折起，以及列车时刻表、50000多旅游景点的旅游攻略信息，为您的度假提供最佳的旅游线路。',
  'category': '旅行',
  'home': 'http://www.kuxun.cn/'},
 {'id': '2246',
  'name': '金山词霸',
  'logo': 'http://reg007.u.qiniudn.com/app/iciba.jpg',
  'desc': '爱词霸英语为广大网友提供在线翻译、在线词典、英语口语、英语学习资料、汉语词典，金山词霸下载等服务，致力于为您提供优质权威的在线英语服务，是5000万英语学习者的首选。',
  'category': '工具',
  'home': 'http://www.iciba.com/'},
 {'id': '2247',
  'name': '一听音乐网',
  '

In [4]:
pattern = re.compile("http://(.+?)/")
url_list = [re.findall(pattern, dic["home"])[0] for dic in website_list]

len(url_list)
url_list[:10]

212

['www.renren.com',
 'www.tuniu.com',
 'www.kuxun.cn',
 'www.iciba.com',
 'www.1ting.com',
 'www.youku.com',
 'www.zol.com.cn',
 'www.baihe.com',
 'www.izhenxin.com',
 'www.taobao.com']

In [4]:
with open("./links.txt", "r", encoding="utf8") as f:
    links=f.read().splitlines()
    
links[:5]

['https://github.com',
 'https://docs.github.com',
 'https://skills.github.com',
 'https://resources.github.com',
 'https://support.github.com']

In [12]:
url_list = [link.split("//")[1].split("/")[0] for link in links]

len(url_list)
url_list[:10]

2921

['github.com',
 'docs.github.com',
 'skills.github.com',
 'resources.github.com',
 'support.github.com',
 'githubstatus.com',
 'twitter.com',
 'partner.github.com',
 'desktop.github.com',
 'services.github.com']

In [14]:
url = random.choice(url_list)
delay = ping(url)

url
delay

'www.tmrow.com'

0.018360614776611328

In [16]:
delete_url=[]
for url in url_list:
    try:
        if not ping(url):
            delete_url.append(url)
    except Exception:
        delete_url.append(url)
        
for url in delete_url:
    url_list.remove(url)

len(url_list)

2385

In [17]:
with open("./links_filtered.json", "w") as f:
    json.dump(url_list, f)

In [2]:
with open("./links_filtered.json", "r", encoding="utf8") as f:
    url_list = json.load(f)
    
len(url_list)
url_list[:5]

2385

['docs.github.com',
 'skills.github.com',
 'resources.github.com',
 'support.github.com',
 'githubstatus.com']

In [3]:
def gen_addr():
    # 生成一个随机的密钥
    while True:
        # 生成一个用十六进制表示的长 256 位的私钥（str类型）
        private_key = bitcoin.random_key()
        # 解码为十进制的整形密钥
        decoded_private_key = bitcoin.decode_privkey(private_key, "hex")
        if 0 < decoded_private_key < bitcoin.N:
            break
    # print(private_key) #密钥（十六进制,长 256 位）

    # 计算地址
    # 用 WIF 格式编码密钥
    wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, "wif")
    # 用 01 标识的压缩密钥
    compressed_private_key = private_key + "01"
    # 生成 WIF的压缩格式
    wif_compressed_private_key = bitcoin.encode_privkey(
        bitcoin.decode_privkey(compressed_private_key, "hex"), "wif"
    )
    # 计算公钥坐标 K = k * G
    public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
    # 计算公钥
    hex_encoded_public_key = bitcoin.encode_pubkey(public_key, "hex")
    # 计算压缩公钥
    # if public_key[1] % 2 == 0:  # 两种方式均可
    if public_key[1] & 1 == 0:
        compressed_prefix = "02"
    else:
        compressed_prefix = "03"
    # 转十六也可用 bitcoin.encode(xxx, 16)
    hex_compressed_public_key = compressed_prefix + hex(public_key[0])[2:]
    # print(f'压缩公钥（十六进制）{hex_compressed_public_key} '
    #      '（02 开头代表 y 是偶数，03 开头代表 y 是奇数）')
    # 传入公钥坐标对象/十六进制公钥值，输出同样的地址
    address = bitcoin.pubkey_to_address(public_key)
    # print(address) #地址（b58check）（1 开头
    return address

In [4]:
# df_user = pd.DataFrame(columns=["user_id", "address", "url"])
# for i in range(2000):
#     url = random.choice(url_list)
#     # while not ping(url):
#     #     url=random.choice(url_list)
#     df_user.loc[i] = [i, gen_addr(), url]

# df_user

,user_id,address,url
0,0,15HrM3t2EKAkxkF9ATF2PVNhFvzXhtHYEf,www.explainshell.com
1,1,17W2zkm5iHtL1NjgVqwS4ircPM6293ATfy,hr.lagou.com
2,2,1CMF9NBpDn8ewgTZP5VA22FLP6fscsyjba,unix.stackexchange.com
3,3,18KKmJaM7XBbSpSqCW8HTN6kz51hgVourt,pedestrianobservations.com
4,4,13wuAwBTDXzDULR7kXg3NJRVWcm1SRKmRP,echarts.baidu.com
...,...,...,...
1995,1995,1M4AS1LXCeDtF1MphV1U6YZP7nkE9QxbPi,www.avabodh.com
1996,1996,18rTX2UwaNgco9858Tk8Xv2eZJkTUUCv82,www.howtogeek.com
1997,1997,1PYfc9fvBRGUPxS7L4HArchsq55D4WmBXG,therottenapple.substack.com
1998,1998,1GXTNHbtrU556U68YqMeLL6F4vm3iDFmvf,books.halfrost.com


In [5]:
df_user = pd.DataFrame(columns=["user_id", "address"])
for i in range(2000):
    df_user.loc[i] = [i, gen_addr()]

df_user["url"]=np.random.permutation(url_list)[:2000]
df_user

,user_id,address,url
0,0,1ND2xK7anee9jF9fxFsR9VH7b4bBSo6W3Y,seawatergreenhouse.com
1,1,1FAJEi9pSK4RchgtqqvdNrMyZNyBxsdizG,www.rbth.com
2,2,1BxeW3EiCi4dnqxkvc7aWiwYXBxXKZEovf,excalidraw.com
3,3,1NMj2F8WoX8oMr94Y8GsSYqRiyFVHL1848,developers.cloudflare.com
4,4,19FD6AuhCT9q55QjejPbEeXGS98Fc4pNzi,priceonomics.com
...,...,...,...
1995,1995,1PtXigdrD22BFfsCp66yjvv8h5fZpu9duV,www.resilience.org
1996,1996,1BePZdC3LNnHNJBnk4z9nPFeRaqVKdGw3w,paleotronic.com
1997,1997,19qGwbWrtA8Ea6oJLPRRAim98NegEn6acS,www.quaxio.com
1998,1998,127chB8vPk8jjdUSWUGDkgh7nkVuhiPmCD,blog.pragmaticengineer.com


In [6]:
df_user.to_csv("user.csv", index=False)

---

In [1]:
!python gen_tx.py > gen_data.log 2>&1

^C


In [4]:
df_tx = pd.read_pickle("./tx_100000.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-11-03 12:06:01.754224,2022-11-03 12:06:01.769617,0.015393,0,1L1DPq9bLnWCWVpoDgknjyyHNUMZeCbiQM,1PQsvHafKbhrySZ24GJ5nb1kNgPKjBwkap,3.652435
1,2022-11-03 12:06:01.756224,2022-11-03 12:06:01.772982,0.016758,0,1AN9ybCX7XWD9kJu7of3R6d7RYEuzKcNRF,1QJVHFQFRhqko4WKoSxd9MihUWcx3YYxNB,9.729553
2,2022-11-03 12:06:01.757224,2022-11-03 12:06:01.774241,0.017017,0,1PxwEuKxveUzGCxHSjSHkdmUqNwpiRQMDG,17PZrJJ4RNcNQ7mLsrZTVcT6PrNWszrvkP,9.799622
3,2022-11-03 12:06:01.766223,2022-11-03 12:06:01.774462,0.008239,0,1KqAuGA7orHFdk85GRqZtZ6fbji1FhWD9N,1EhQWGGxHkCyQMQmokRiAPJrPcCUxCDuzc,7.831164
4,2022-11-03 12:06:01.760223,2022-11-03 12:06:01.778436,0.018213,1,15HVHd2YTPm5v4rzeu74BBW3ZFUA2B1sYa,1FnEcqny69RbvKHVveNrN81s1K3zui8dfp,0.388215
...,...,...,...,...,...,...,...
99995,2022-11-03 12:41:17.399719,2022-11-03 12:41:17.616094,0.216375,0,1GDGhgbp9zYb8zQCKfRppa6mH5LnX88iVg,1HfyC6biqQhqs8ws9JBQqmxkzxJMydE1Rj,5.933808
99996,2022-11-03 12:41:17.384719,2022-11-03 12:41:17.626464,0.241745,1,13TpvfWFetXGGMsFdVqvycVanLTPLU7V6Q,18SaEeetM2rNTXU968eVxZiQkjLS41o3BQ,4.851685
99997,2022-11-03 12:41:17.396719,2022-11-03 12:41:17.628146,0.231427,0,15iPQwtatv7NDCvRYrQDg6bK2nZnSM9faC,14uBqhNEW5uhaw9dKEpeb6QqzGtYJCaHBL,5.851299
99998,2022-11-03 12:41:17.334718,2022-11-03 12:41:17.628326,0.293608,1,1hEPNpR4g2mha82hCTXiwRX7eJWLHsdE2,1EV5jaKb1hSPvY8DxPw4e5e234Qxjv4Qtv,7.671977


In [5]:
sum(df_tx["label"] == 1)

48301

---

---

Generate training data

In [3]:
df_tx = pd.read_pickle("./tx_100000.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-11-03 12:06:01.754224,2022-11-03 12:06:01.769617,0.015393,0,1L1DPq9bLnWCWVpoDgknjyyHNUMZeCbiQM,1PQsvHafKbhrySZ24GJ5nb1kNgPKjBwkap,3.652435
1,2022-11-03 12:06:01.756224,2022-11-03 12:06:01.772982,0.016758,0,1AN9ybCX7XWD9kJu7of3R6d7RYEuzKcNRF,1QJVHFQFRhqko4WKoSxd9MihUWcx3YYxNB,9.729553
2,2022-11-03 12:06:01.757224,2022-11-03 12:06:01.774241,0.017017,0,1PxwEuKxveUzGCxHSjSHkdmUqNwpiRQMDG,17PZrJJ4RNcNQ7mLsrZTVcT6PrNWszrvkP,9.799622
3,2022-11-03 12:06:01.766223,2022-11-03 12:06:01.774462,0.008239,0,1KqAuGA7orHFdk85GRqZtZ6fbji1FhWD9N,1EhQWGGxHkCyQMQmokRiAPJrPcCUxCDuzc,7.831164
4,2022-11-03 12:06:01.760223,2022-11-03 12:06:01.778436,0.018213,1,15HVHd2YTPm5v4rzeu74BBW3ZFUA2B1sYa,1FnEcqny69RbvKHVveNrN81s1K3zui8dfp,0.388215
...,...,...,...,...,...,...,...
99995,2022-11-03 12:41:17.399719,2022-11-03 12:41:17.616094,0.216375,0,1GDGhgbp9zYb8zQCKfRppa6mH5LnX88iVg,1HfyC6biqQhqs8ws9JBQqmxkzxJMydE1Rj,5.933808
99996,2022-11-03 12:41:17.384719,2022-11-03 12:41:17.626464,0.241745,1,13TpvfWFetXGGMsFdVqvycVanLTPLU7V6Q,18SaEeetM2rNTXU968eVxZiQkjLS41o3BQ,4.851685
99997,2022-11-03 12:41:17.396719,2022-11-03 12:41:17.628146,0.231427,0,15iPQwtatv7NDCvRYrQDg6bK2nZnSM9faC,14uBqhNEW5uhaw9dKEpeb6QqzGtYJCaHBL,5.851299
99998,2022-11-03 12:41:17.334718,2022-11-03 12:41:17.628326,0.293608,1,1hEPNpR4g2mha82hCTXiwRX7eJWLHsdE2,1EV5jaKb1hSPvY8DxPw4e5e234Qxjv4Qtv,7.671977


In [6]:
df_user=pd.read_csv("./user.csv")

user_list=df_user["address"].values.tolist()

user_list[:5]

['1ND2xK7anee9jF9fxFsR9VH7b4bBSo6W3Y',
 '1FAJEi9pSK4RchgtqqvdNrMyZNyBxsdizG',
 '1BxeW3EiCi4dnqxkvc7aWiwYXBxXKZEovf',
 '1NMj2F8WoX8oMr94Y8GsSYqRiyFVHL1848',
 '19FD6AuhCT9q55QjejPbEeXGS98Fc4pNzi']

In [7]:
def get_user_id(row):
    from_user_id=user_list.index(row["from"])
    to_user_id=user_list.index(row["to"])
    
    row["from_user_id"]=from_user_id
    row["to_user_id"]=to_user_id
    
    return row

In [8]:
df_data=df_tx.copy()
df_data=df_data.apply(get_user_id, axis=1)

df_data

,send_timestamp,recv_timestamp,latency,label,from,to,value,from_user_id,to_user_id
0,2022-11-03 12:06:01.754224,2022-11-03 12:06:01.769617,0.015393,0,1L1DPq9bLnWCWVpoDgknjyyHNUMZeCbiQM,1PQsvHafKbhrySZ24GJ5nb1kNgPKjBwkap,3.652435,778,675
1,2022-11-03 12:06:01.756224,2022-11-03 12:06:01.772982,0.016758,0,1AN9ybCX7XWD9kJu7of3R6d7RYEuzKcNRF,1QJVHFQFRhqko4WKoSxd9MihUWcx3YYxNB,9.729553,31,1003
2,2022-11-03 12:06:01.757224,2022-11-03 12:06:01.774241,0.017017,0,1PxwEuKxveUzGCxHSjSHkdmUqNwpiRQMDG,17PZrJJ4RNcNQ7mLsrZTVcT6PrNWszrvkP,9.799622,714,1731
3,2022-11-03 12:06:01.766223,2022-11-03 12:06:01.774462,0.008239,0,1KqAuGA7orHFdk85GRqZtZ6fbji1FhWD9N,1EhQWGGxHkCyQMQmokRiAPJrPcCUxCDuzc,7.831164,1550,824
4,2022-11-03 12:06:01.760223,2022-11-03 12:06:01.778436,0.018213,1,15HVHd2YTPm5v4rzeu74BBW3ZFUA2B1sYa,1FnEcqny69RbvKHVveNrN81s1K3zui8dfp,0.388215,1684,377
...,...,...,...,...,...,...,...,...,...
99995,2022-11-03 12:41:17.399719,2022-11-03 12:41:17.616094,0.216375,0,1GDGhgbp9zYb8zQCKfRppa6mH5LnX88iVg,1HfyC6biqQhqs8ws9JBQqmxkzxJMydE1Rj,5.933808,1510,1185
99996,2022-11-03 12:41:17.384719,2022-11-03 12:41:17.626464,0.241745,1,13TpvfWFetXGGMsFdVqvycVanLTPLU7V6Q,18SaEeetM2rNTXU968eVxZiQkjLS41o3BQ,4.851685,1625,399
99997,2022-11-03 12:41:17.396719,2022-11-03 12:41:17.628146,0.231427,0,15iPQwtatv7NDCvRYrQDg6bK2nZnSM9faC,14uBqhNEW5uhaw9dKEpeb6QqzGtYJCaHBL,5.851299,304,1027
99998,2022-11-03 12:41:17.334718,2022-11-03 12:41:17.628326,0.293608,1,1hEPNpR4g2mha82hCTXiwRX7eJWLHsdE2,1EV5jaKb1hSPvY8DxPw4e5e234Qxjv4Qtv,7.671977,299,664


In [9]:
df_data.drop(columns=["send_timestamp", "from", "to"], inplace=True)

df_data

,recv_timestamp,latency,label,value,from_user_id,to_user_id
0,2022-11-03 12:06:01.769617,0.015393,0,3.652435,778,675
1,2022-11-03 12:06:01.772982,0.016758,0,9.729553,31,1003
2,2022-11-03 12:06:01.774241,0.017017,0,9.799622,714,1731
3,2022-11-03 12:06:01.774462,0.008239,0,7.831164,1550,824
4,2022-11-03 12:06:01.778436,0.018213,1,0.388215,1684,377
...,...,...,...,...,...,...
99995,2022-11-03 12:41:17.616094,0.216375,0,5.933808,1510,1185
99996,2022-11-03 12:41:17.626464,0.241745,1,4.851685,1625,399
99997,2022-11-03 12:41:17.628146,0.231427,0,5.851299,304,1027
99998,2022-11-03 12:41:17.628326,0.293608,1,7.671977,299,664


In [20]:
# df_data["recv_timestamp"]=df_data["recv_timestamp"].apply(lambda x: x.value)

# df_data

In [10]:
df_data.to_pickle(f"./data_{len(df_data)}.pkl")

In [22]:
df_data[["recv_timestamp", "from_user_id", "to_user_id", "value", "label"]].values[:5]

array([[Timestamp('2022-10-25 09:48:30.432322'), 1907, 227,
        5.112375488707563, 0],
       [Timestamp('2022-10-25 09:48:30.434821'), 60, 1090,
        8.10742405244109, 1],
       [Timestamp('2022-10-25 09:48:30.435350'), 1583, 1241,
        0.26199133937441, 0],
       [Timestamp('2022-10-25 09:48:30.478155'), 1514, 1682,
        0.8488239081643234, 0],
       [Timestamp('2022-10-25 09:48:30.526710'), 1112, 900,
        1.8435082598156494, 1]], dtype=object)